In [ ]:
import os

def rename_files_in_folder(folder_path):
    # 이미지 파일 확장자 리스트
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']

    # 폴더 내의 모든 파일을 확인
    for filename in os.listdir(folder_path):
        # 파일 확장자를 확인하여 이미지 파일인지 확인
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            # 파일명에 'bg'가 포함되지 않은 경우
            if 'bg' not in filename:
                # 새로운 파일명 생성
                new_filename = 'char ' + filename
                # 파일 경로 생성
                old_file = os.path.join(folder_path, filename)
                new_file = os.path.join(folder_path, new_filename)
                # 파일명 변경
                os.rename(old_file, new_file)
                print(f'Renamed: {filename} to {new_filename}')

# 사용 예시: 'path_to_your_folder'을 실제 폴더 경로로 변경하세요
folder_path = r'C:\Users\highl\Desktop\RENPY\renpyGame\game\images'
rename_files_in_folder(folder_path)


In [14]:
import csv
import re
import os

char_name_dict = { 'S': 's', 'T': 'T', 'H': 'h', 'N': 'n', 'E': 'e', 'A': 'a'}


def get_char_id(char_name):
    return char_name_dict.get(char_name, f"Character('{char_name}')")

def replace_underscore_with_space(text):
    return text.replace('_', ' ')

def csv_to_renpy(csv_file, rpy_file, chapter_num = 0):
    menu_count = 0
    is_showing_char = False
    is_in_jump1 = False
    choise_Q = []
    teb_count = 0
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        lines = list(reader)
        

    with open(rpy_file, 'w', encoding='utf-8') as file:
        file.write(f"label chapter_{chapter_num}:\n")
        
        for line in lines:
            if line['Chrt_Pos']:
                char_onf = replace_underscore_with_space(line['Chrt_Pos'].lower())
                #showing char image
                file.write(('    ' * teb_count) + f"    show char {char_onf} at top\n")
                is_showing_char = True
                
            if (l := line['Event']) and (t := line['Text']):
                if l == 'menu':
                    file.write(f"    menu .m{menu_count}:\n")
                    menu_count += 1
                    teb_count = 1
                elif l.lower() == 'choise1' or l.lower() == 'choise2':
                    choise_Q.append(f"        \"{t}\":\n")
                    teb_count = 2
                    is_in_jump1 = True
                    continue
                elif l == 'jump1' or l == 'jump2':
                    file.write(choise_Q.pop(0))
                elif l.lower() == 'out' or l.lower() == 'end':
                    if (l := len(choise_Q)) >= 0:
                        #print("pop all choise_Q")
                        for t in range(l):
                            f = choise_Q.pop(0)
                            file.write(f)
                            file.write("    " + f[:-2] + "\n")
                            
                    teb_count = 0
                    

            # background
            if line['background']:
                background = replace_underscore_with_space(line['background'].lower())
                if background == 'black' or background == 'white':
                    file.write(('    ' * teb_count) + f"    scene {background}\n")
                else:
                    file.write(('    ' * teb_count) + f"    scene bg {background}\n")
            
            # Sound
            if line['sound']:
                file.write(('    ' * teb_count) + f"    play sound \"sfx/{line['sound'].lower()}.mp3\"\n")
            
            # BGM
            if line['BGM']:
                file.write(('    ' * teb_count) + f"    play music \"bgm/{line['BGM'].lower()}.mp3\"\n")
            
            # Character and dialogue
            if line['Chrt_ID'] and line['Text']:
                char_id = replace_underscore_with_space(line['Chrt_ID'])

                if char_id == '???':
                    char_id = 'q'
                char_id = get_char_id(char_id)
                
                
                file.write(('    ' * teb_count) + f"    {char_id} \"{line['Text']}\"\n")
            elif line['Text']:
                # Narration
                narration = re.sub(r'\(|\)', '', line['Text'])  # Remove parentheses

                if is_showing_char:
                    file.write(('    ' * teb_count) + "    hide char\n" + ('    ' * teb_count) + f"    \"{narration}\"\n")
                else:
                    file.write(('    ' * teb_count) + f"    \"{narration}\"\n")
                is_showing_char = False
                
                
            
            # Event (e.g., shake)
            if l:=line['Event']:
                if l == 'menu' or l == 'Out' or l == 'jump1' or l == 'jump2' or l == 'end':
                    continue
                if l == 'red' or l == 'white':
                    file.write(('    ' * teb_count) + f"    show {line['Event']} at fadeInOut\n")
                    continue
                file.write(('    ' * teb_count) + f"    with {line['Event']}\n")
            
            file.write("\n")
        
        file.write("    return\n")


In [16]:
def find_files_in_directory(directory, extension=None):
    files = []
    for filename in os.listdir(directory):
        if extension:
            if filename.endswith(extension):
                files.append(filename)
        else:
            files.append(filename)
    return files

In [ ]:
# 스크립트 파일의 디렉토리를 기준으로 경로 설정
current_dir = os.getcwd()
input_file = os.path.join(current_dir, 'input.csv')
output_file = os.path.join(current_dir, 'output.rpy')
# Usage
files = find_files_in_directory(current_dir)
files.sort()
for i in len(files):
    csv_to_renpy(os.path.join(current_dir, files[i]), f'chapter{i}.rpy',i)
    print(f"chapter {i} done.")
print('all done')

In [8]:
#csv_to_renpy(r'D:\renpyGame\csvs\Chapter_0.csv',r'D:\renpyGame\rpys\Chapter_0.rpy')
n = 14

filename = f'Chapter_{n}'
inputfile = 'D:\\renpyGame\\csvs\\' + filename + '.csv'
outputfile = 'D:\\renpyGame\\rpys\\' + filename + '.rpy'
print(f'work {filename}')
try:
    csv_to_renpy(inputfile,outputfile)
except Exception as e:
    print(f"오류 발생: {e}")
    i = input("다시 실행할까요? no = 'n'")
    if i == 'n':
        pass
    else:
        pass
else:
    print(f"{filename} done.")
    n += 1

work Chapter_14
Chapter_14 done.


In [18]:
csv_to_renpy(r'D:\renpyGame\csvs\Ending_Nox.csv',r'D:\renpyGame\rpys\Ending_Nox.rpy')